<div id="colab_button">
  <h1>Quick tour</h1>
  <a target="_blank" href="https://colab.research.google.com/github/mithril-security/blindai/blob/main/docs/docs/getting-started/quick-tour.ipynb"> 
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
</div>
______________________________

In this notebook, we will explore how **BlindAI** can be used to **query popular AI models** while offering **strong privacy guarantees**. It does so by processing user data in a **SGX secure enclave**.

To let you discover BlindAI without having to install anything locally, we made [**a Google Colab**](https://colab.research.google.com/github/mithril-security/blindai/blob/main/docs/docs/getting-started/quick-tour.ipynb) where you can directly follow this introduction tutorial with or without a Google account. However, everything has been explained and packaged for you to be able to run tests on your machine, if you wish to.

> If you're still wondering why you should use BlindAI, you can go check [our introduction](https://blindai.mithrilsecurity.io/en/latest/docs/getting-started/why-blindai/) on that topic. We'll focus on a hands-on demo in this tutorial.


### The use case: Secure audio transcription

In this tutorial, we take the use case of a hospital that wants to automate transcriptions of consultations with patients. This would help doctors to make a record of everything covered in consultations which they can refer back to at any time, saving the need for them to take time out to write up lengthy notes.

There are lots of existing speech-to-text applications, but the hospital has to be very careful- this is highly sensitive and critical patient data. They need to be sure that their data cannot be accessed or tampered by any third party, whether that be the speech-to-text application creator, the application's Cloud provider or a malicious hacker targeting one of these... And that's where BlindAI comes in!

BlindAI offers secure audio transcription using the Whisper model, an automatic speech recognition (ASR) system trained on 680,000 hours of multilingual and multitask supervised data collected from the web. Data sent to any of the models available on BlindAI is kept confidential at all times- no one can access or tamper with the data. You can learn more about how user data is protected [here](LIEN).


So with that said, let's take a look at how to set-up and use BlindAI.

## Pre-requisites
________________________________________________________

Before we begin, you will need to install the [blindai](https://pypi.org/project/blindai/) Python library and download a [test audio file](https://www2.cs.uic.edu/~i101/SoundFiles/taunt.wav) provided by University of Illinois at Chicago  as part of their Computer Science 101 course.

In [1]:
# install blindai
!pip install blindai

# Download our example audio file and save it as `taunt.wav`
!wget https://www2.cs.uic.edu/~i101/SoundFiles/taunt.wav -O taunt.wav

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.1/223.1 KB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

--2023-03-30 19:51:03--  https://www2.cs.uic.edu/~i101/SoundFiles/taunt.wav
Resolving www2.cs.uic.edu (www2.cs.uic.edu)... 131.193.32.16
Connecting to www2.cs.uic.edu (www2.cs.uic.edu)|131.193.32.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 91284 (89K) [audio/x-wav]
Saving to: ‘taunt.wav’

taunt.wav           100%[===================>]  89.14K   176KB/s    in 0.5s    

2023-03-30 19:51:05 (176 KB/s) - ‘taunt.wav’ saved [91284/91284]



## Transcribing our audio file
______________________________________________

Speech-to-text operations are handled within the `Audio.transcribe` module of `BlindAI.api`.

To transcribe an audio file, we call the `transcribe` method and provide the path to our audio file as the `file` option. The Whisper model accepts a variety of audio file formats (m4a, mp3, mp4, mpeg, mpga, wav, webm).

In [2]:
import blindai

transcript = blindai.api.Audio.transcribe("taunt.wav")

>If you or your service provider are running your own instance of the BlindAI server and you don't wish to connect to our managed cloud solution, you can pass your `BlindAiConnection` client object to the `transcribe` method's `connection` option.

We can print out the returned `transcript` string variable to check our results are as expected.

In [3]:
print(transcript)

 Now go away, or I shall taunt you a second timer!


Our audio file has been correctly transcribed. Feel free to test this out with your own audio files!


## Under the hood
______________________________________________

Let's take a moment to discuss what went on under the hood when we called the `blindai.api.Audio.transcribe` method.

### Connection

Firstly, the `transcribe()` method connects to our managed cloud solution by calling the BlindAI.Core's `connect()` method, like so:

In [4]:
DEFAULT_BLINDAI_ADDR = "4.246.205.63"

connection = blindai.core.connect(
            DEFAULT_BLINDAI_ADDR,
            hazmat_http_on_unattested_port=True,
            use_cloud_manifest=True,
        )

It is at this point that the attestation process described in our [confidential computing guide](confidential_computing.md) is triggered.

This process will check that the Mithril server instance is running a real Trusted Execution Environment (TEE), which is a highly isolated computing environment, using genuine hardware. 

The client also receives a hash of the application code running in the TEE which it will check exactly matches the code for the latest release of BlindAI which is defined in the client's built-in `manifest.toml` file. This is to prevent users being able to connect with a version of the application which has been tampered with and may not be trustworthy.

We can quickly prove that this check is in place by doing the following:
1. Creating an empty `manifest.toml` file called `fake_manifest.toml`
2. Using the `connect()` method's `hazmat_manifest_path` option to override the default path for the `manifest.toml` file with the path for our `fake_manifest.toml` file

Uncomment and run the following code to see this for yourself.

In [6]:
# !touch fake_manifest.toml

# connection = blindai.core.connect(
#             DEFAULT_BLINDAI_ADDR,
#             hazmat_http_on_unattested_port=True,
#             use_cloud_manifest=True,
#             hazmat_manifest_path="./fake_manifest.toml"
#         )

AttestationError: ignored

This causes the connection to be refused with an `AttestationError` because the application code cannot be verified.

> Note, to test this more thoroughly, you can download our Github project and modify the server code stored in the `src` folder very slightly. You then need to generate a new `manifest.toml` based on this modified source code by building the project. You can then replace the path to the `fake_manifest.toml` in the previous example with a path to your newly generated `manifest.toml`. This will lead to the same error, since the server's hash received in the attestation report will not match the code in the newly generated `manifest.toml`.

### Querying the model

Once connected, the `transcribe` method will then prepare the audio file correctly for the model and then call the BlindAI Core `run_model` method, passing it the ID for the default Whisper model and the now pre-processed input data:

```py
res = connection.run_model(model_hash=DEFAULT_WHISPER_MODEL, input_tensors=input_data)
```

Then, the method simply extracts the text transcription from the returned object and returns that to our BlindAI API user!

```
# This is formatted as code
```




### Conclusions

This is the end of our introduction to BlindAI!

We have seen:
- How to transcribe audio using the Whisper model in BlindAI
- How this works under the hood

Please check out the rest of our BlindAI documentation to see more examples of how you can use BlindAI to query AI models without compromising the safety of user data or models.